In [28]:
#Import data
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib
import numpy as nps
from geopy.extra.rate_limiter import RateLimiter
from geopy.geocoders import Nominatim
import folium
from sklearn.cluster import KMeans ,AgglomerativeClustering
import gensim

In [29]:
#Read data
data = pd.read_excel('MSdata.xlsx') 
print(data.head())

month  TermOfPaymentDays  Amount_netCN  DefaultStatus  Age    ZIP  Country  \
0     12                110        204.59              0   74  52152  Germany   
1     12                 30         29.90              0   57  44339  Germany   
2     12                140        229.49              0   45  84095  Germany   
3     12                140        264.48              0   76  99734  Germany   
4     12                 80        145.68              0   53  14727  Germany   

                    State  GDP by State  Unemployment rate  
0  North Rhine-Westphalia       39400.0              0.066  
1  North Rhine-Westphalia       39400.0              0.066  
2                 Bavaria       47900.0              0.027  
3               Thuringia       29700.0              0.052  
4             Brandenburg       29400.0              0.057  


In [30]:
#Get info
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 327051 entries, 0 to 327050
Data columns (total 10 columns):
month                327051 non-null int64
TermOfPaymentDays    327051 non-null int64
Amount_netCN         327051 non-null float64
DefaultStatus        327051 non-null int64
Age                  327051 non-null int64
ZIP                  327051 non-null object
Country              327051 non-null object
State                325626 non-null object
GDP by State         325626 non-null float64
Unemployment rate    325626 non-null float64
dtypes: float64(3), int64(4), object(3)
memory usage: 25.0+ MB


In [31]:
#Drop Null rows
data.dropna(subset=['State'], inplace=True)
data.reset_index(drop=True)

#check null values again
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325626 entries, 0 to 327050
Data columns (total 10 columns):
month                325626 non-null int64
TermOfPaymentDays    325626 non-null int64
Amount_netCN         325626 non-null float64
DefaultStatus        325626 non-null int64
Age                  325626 non-null int64
ZIP                  325626 non-null object
Country              325626 non-null object
State                325626 non-null object
GDP by State         325626 non-null float64
Unemployment rate    325626 non-null float64
dtypes: float64(3), int64(4), object(3)
memory usage: 27.3+ MB


In [32]:
#Data with all needed features
Data_clean = data[['month', "Age","Amount_netCN", "TermOfPaymentDays", 'GDP by State', 'Unemployment rate', 'ZIP', "DefaultStatus", 'State' ]]
#get info
Data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325626 entries, 0 to 327050
Data columns (total 9 columns):
month                325626 non-null int64
Age                  325626 non-null int64
Amount_netCN         325626 non-null float64
TermOfPaymentDays    325626 non-null int64
GDP by State         325626 non-null float64
Unemployment rate    325626 non-null float64
ZIP                  325626 non-null object
DefaultStatus        325626 non-null int64
State                325626 non-null object
dtypes: float64(3), int64(4), object(2)
memory usage: 24.8+ MB


In [33]:
# Check null value (Print a null value column array)
print(Data_clean.columns[Data_clean.isnull().any()])

Index([], dtype='object')


In [34]:
#Save data
Data_clean.to_excel('data_dropNull.xlsx', index = False)

In [35]:
#Data with all needed features
Data_clean = data[['month', "Age","Amount_netCN", "TermOfPaymentDays", 'GDP by State', 'Unemployment rate', 'ZIP', "DefaultStatus" ]]
#get info
Data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325626 entries, 0 to 327050
Data columns (total 8 columns):
month                325626 non-null int64
Age                  325626 non-null int64
Amount_netCN         325626 non-null float64
TermOfPaymentDays    325626 non-null int64
GDP by State         325626 non-null float64
Unemployment rate    325626 non-null float64
ZIP                  325626 non-null object
DefaultStatus        325626 non-null int64
dtypes: float64(3), int64(4), object(1)
memory usage: 22.4+ MB


In [36]:
#Creating dummy variables
month = pd.get_dummies(Data_clean['month'])
len(month.columns)
name_month = []
for i in range(1,len(month.columns)+1):
    name_month.append('month'+str(i))
#rename colum   
month.columns = name_month

In [37]:
#Concatenating the dummy variables to the original dataset 
Data_clean=pd.concat([Data_clean,month],axis=1)
#show data
Data_clean.head(10)

,month,Age,Amount_netCN,TermOfPaymentDays,GDP by State,Unemployment rate,ZIP,DefaultStatus,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12
0,12,74,204.59,110,39400.0,0.066,52152,0,0,0,0,0,0,0,0,0,0,0,0,1
1,12,57,29.90,30,39400.0,0.066,44339,0,0,0,0,0,0,0,0,0,0,0,0,1
2,12,45,229.49,140,47900.0,0.027,84095,0,0,0,0,0,0,0,0,0,0,0,0,1
3,12,76,264.48,140,29700.0,0.052,99734,0,0,0,0,0,0,0,0,0,0,0,0,1
4,12,53,145.68,80,29400.0,0.057,14727,0,0,0,0,0,0,0,0,0,0,0,0,1
5,12,74,204.59,110,47900.0,0.027,95445,0,0,0,0,0,0,0,0,0,0,0,0,1
6,12,49,249.50,140,47900.0,0.027,80638,0,0,0,0,0,0,0,0,0,0,0,0,1
7,12,55,264.48,140,47900.0,0.027,96132,0,0,0,0,0,0,0,0,0,0,0,0,1
8,12,51,204.59,110,27900.0,0.067,19061,0,0,0,0,0,0,0,0,0,0,0,0,1
9,12,75,204.59,110,36600.0,0.045,67240,0,0,0,0,0,0,0,0,0,0,0,0,1


In [38]:
#get info
Data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325626 entries, 0 to 327050
Data columns (total 20 columns):
month                325626 non-null int64
Age                  325626 non-null int64
Amount_netCN         325626 non-null float64
TermOfPaymentDays    325626 non-null int64
GDP by State         325626 non-null float64
Unemployment rate    325626 non-null float64
ZIP                  325626 non-null object
DefaultStatus        325626 non-null int64
month1               325626 non-null uint8
month2               325626 non-null uint8
month3               325626 non-null uint8
month4               325626 non-null uint8
month5               325626 non-null uint8
month6               325626 non-null uint8
month7               325626 non-null uint8
month8               325626 non-null uint8
month9               325626 non-null uint8
month10              325626 non-null uint8
month11              325626 non-null uint8
month12              325626 non-null uint8
dtypes: float64(3), in

In [39]:
# Create a cross table of the ZIP and DefaultStatus
ZipCode =pd.crosstab(Data_clean['ZIP'], Data_clean['DefaultStatus'], margins = True)
ZipCode

DefaultStatus,0,1,All
ZIP,,,
1067,46,2,48
1069,79,8,87
1097,41,1,42
1099,65,3,68
1108,19,0,19
...,...,...,...
99991,33,2,35
99994,23,1,24
99996,25,1,26


In [40]:
#Create Feq_default_ZipCode feature
ZipCode['Feq_default_Country']=ZipCode[1]/ZipCode[1].sum()
ZipCode['Feq_default_ZipCode']=ZipCode[1]/ZipCode['All']
ZipCode.sort_values(by=['Feq_default_ZipCode'], ascending=False)

DefaultStatus,0,1,All,Feq_default_Country,Feq_default_ZipCode
ZIP,,,,,
57645,0,1,1,0.000024,1.0
54612,0,1,1,0.000024,1.0
25593,0,1,1,0.000024,1.0
21527,0,1,1,0.000024,1.0
35687,0,1,1,0.000024,1.0
...,...,...,...,...,...
83543,29,0,29,0.000000,0.0
83544,3,0,3,0.000000,0.0
83546,1,0,1,0.000000,0.0


In [41]:
#Create Feq_default_ZipCode feature
ZipCode.drop(columns=[0, 1, 'All', 'Feq_default_Country'], inplace= True)

In [42]:
#get info
ZipCode.info()

<class 'pandas.core.frame.DataFrame'>
Index: 8112 entries, 1067 to All
Data columns (total 1 columns):
Feq_default_ZipCode    8112 non-null float64
dtypes: float64(1)
memory usage: 126.8+ KB


In [43]:
#Merge data
Data_clean = pd.merge(Data_clean, ZipCode['Feq_default_ZipCode'], on='ZIP')
#get info
Data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325626 entries, 0 to 325625
Data columns (total 21 columns):
month                  325626 non-null int64
Age                    325626 non-null int64
Amount_netCN           325626 non-null float64
TermOfPaymentDays      325626 non-null int64
GDP by State           325626 non-null float64
Unemployment rate      325626 non-null float64
ZIP                    325626 non-null object
DefaultStatus          325626 non-null int64
month1                 325626 non-null uint8
month2                 325626 non-null uint8
month3                 325626 non-null uint8
month4                 325626 non-null uint8
month5                 325626 non-null uint8
month6                 325626 non-null uint8
month7                 325626 non-null uint8
month8                 325626 non-null uint8
month9                 325626 non-null uint8
month10                325626 non-null uint8
month11                325626 non-null uint8
month12                3256

In [44]:
#Show data
Data_clean.head()

,month,Age,Amount_netCN,TermOfPaymentDays,GDP by State,Unemployment rate,ZIP,DefaultStatus,month1,month2,...,month4,month5,month6,month7,month8,month9,month10,month11,month12,Feq_default_ZipCode
0,12,74,204.59,110,39400.0,0.066,52152,0,0,0,...,0,0,0,0,0,0,0,0,1,0.081633
1,9,72,60.88,30,39400.0,0.066,52152,0,0,0,...,0,0,0,0,0,1,0,0,0,0.081633
2,8,62,160.68,30,39400.0,0.066,52152,0,0,0,...,0,0,0,0,1,0,0,0,0,0.081633
3,8,75,130.77,30,39400.0,0.066,52152,0,0,0,...,0,0,0,0,1,0,0,0,0,0.081633
4,8,68,42.89,30,39400.0,0.066,52152,1,0,0,...,0,0,0,0,1,0,0,0,0,0.081633


In [45]:
#get info
Data_clean.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 325626 entries, 0 to 325625
Data columns (total 21 columns):
month                  325626 non-null int64
Age                    325626 non-null int64
Amount_netCN           325626 non-null float64
TermOfPaymentDays      325626 non-null int64
GDP by State           325626 non-null float64
Unemployment rate      325626 non-null float64
ZIP                    325626 non-null object
DefaultStatus          325626 non-null int64
month1                 325626 non-null uint8
month2                 325626 non-null uint8
month3                 325626 non-null uint8
month4                 325626 non-null uint8
month5                 325626 non-null uint8
month6                 325626 non-null uint8
month7                 325626 non-null uint8
month8                 325626 non-null uint8
month9                 325626 non-null uint8
month10                325626 non-null uint8
month11                325626 non-null uint8
month12                3256

In [46]:
# Check the defaut status values
print(Data_clean['DefaultStatus'].value_counts())

0    304637
1     20989
Name: DefaultStatus, dtype: int64


In [47]:
#Data is imbalanced, so we need to create a balanced data
count_nondefault, count_default = Data_clean['DefaultStatus'].value_counts()

In [48]:
# Create data sets for defaults and non-defaults
nondefaults = Data_clean [Data_clean ['DefaultStatus'] == 0]
defaults = Data_clean [Data_clean ['DefaultStatus'] == 1]

In [49]:
# Undersample the non-defaults
nondefaults_under = nondefaults.sample(count_default)

# Concatenate the undersampled nondefaults with defaults
data_balanced = pd.concat([nondefaults_under.reset_index(drop = True),
                             defaults.reset_index(drop = True)], axis = 0)

# Print the value counts for loan status
print(data_balanced['DefaultStatus'].value_counts())

1    20989
0    20989
Name: DefaultStatus, dtype: int64


In [50]:
#get info
data_balanced.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41978 entries, 0 to 20988
Data columns (total 21 columns):
month                  41978 non-null int64
Age                    41978 non-null int64
Amount_netCN           41978 non-null float64
TermOfPaymentDays      41978 non-null int64
GDP by State           41978 non-null float64
Unemployment rate      41978 non-null float64
ZIP                    41978 non-null object
DefaultStatus          41978 non-null int64
month1                 41978 non-null uint8
month2                 41978 non-null uint8
month3                 41978 non-null uint8
month4                 41978 non-null uint8
month5                 41978 non-null uint8
month6                 41978 non-null uint8
month7                 41978 non-null uint8
month8                 41978 non-null uint8
month9                 41978 non-null uint8
month10                41978 non-null uint8
month11                41978 non-null uint8
month12                41978 non-null uint8
Feq_

In [51]:
data_balanced.describe() 

,month,Age,Amount_netCN,TermOfPaymentDays,GDP by State,Unemployment rate,DefaultStatus,month1,month2,month3,month4,month5,month6,month7,month8,month9,month10,month11,month12,Feq_default_ZipCode
count,41978.000000,41978.000000,41978.000000,41978.000000,41978.000000,41978.000000,41978.000000,41978.000000,41978.000000,41978.000000,41978.000000,41978.000000,41978.000000,41978.000000,41978.000000,41978.000000,41978.000000,41978.000000,41978.000000,41978.000000
mean,4.894111,58.403092,81.827509,35.624327,39889.973319,0.050802,0.500000,0.087189,0.080018,0.118157,0.180738,0.153604,0.151818,0.069536,0.085045,0.040783,0.015413,0.010148,0.007552,0.078552
std,2.392916,15.406551,57.233846,23.996713,7467.256856,0.016262,0.500006,0.282114,0.271324,0.322798,0.384805,0.360573,0.358848,0.254367,0.278951,0.197790,0.123189,0.100227,0.086572,0.053868
min,1.000000,20.000000,9.900000,21.000000,27900.000000,0.027000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,49.000000,50.880000,30.000000,36200.000000,0.031000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046512
50%,5.000000,59.000000,60.880000,30.000000,39400.000000,0.051000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.069767
75%,6.000000,69.000000,84.760000,30.000000,46300.000000,0.066000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000
max,12.000000,101.000000,481.970000,233.000000,65600.000000,0.102000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [52]:
#show data
data_balanced.head()

,month,Age,Amount_netCN,TermOfPaymentDays,GDP by State,Unemployment rate,ZIP,DefaultStatus,month1,month2,...,month4,month5,month6,month7,month8,month9,month10,month11,month12,Feq_default_ZipCode
0,2,66,60.78,30,47900.0,0.027,85077,0,0,1,...,0,0,0,0,0,0,0,0,0,0.062500
1,4,78,50.88,30,47900.0,0.027,81247,0,0,0,...,1,0,0,0,0,0,0,0,0,0.015385
2,9,73,60.88,30,37100.0,0.051,49356,0,0,0,...,0,0,0,0,0,1,0,0,0,0.102564
3,5,51,42.89,30,39400.0,0.066,46284,0,0,0,...,0,1,0,0,0,0,0,0,0,0.111111
4,4,78,59.78,30,47900.0,0.027,84076,0,0,0,...,1,0,0,0,0,0,0,0,0,0.032258


In [53]:
#Save data, ready for tranning models
data_balanced.to_excel('data_balanced.xlsx', index = False)